In [2]:
import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as f
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transform
from torchvision import transforms
import matplotlib.pyplot as plt

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [4]:
train_dir = "D:/KMUTT/Year 4th/ENE490 DL/Coding/Dataset/brain_tumor_dataset/Brain Tumor Classification/Training"
test_dir = "D:/KMUTT/Year 4th/ENE490 DL/Coding/Dataset/brain_tumor_dataset/Brain Tumor Classification/Testing"

In [5]:
train_transform = transforms.Compose([
                                   transforms.Resize((300,300)),
                                   transforms.ToTensor(),
                                   transforms.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5]) # [value-mean]/std
                                   ])

In [6]:
num_classes = 4
learning_rate = 0.001
batch_size = 32
num_epoch = 3

In [7]:
train_data = torchvision.datasets.ImageFolder(root=train_dir,transform = train_transform)
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)



test_data = torchvision.datasets.ImageFolder(root=test_dir,transform = train_transform)
test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

In [8]:
class VGG16C(nn.Module):
    def __init__(self,in_channels=3,num_classes=4):
       super(VGG16C,self).__init__()
       self.conv64_1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)
    #    self.conv64_2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
       self.conv128_1 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
    #    self.conv128_2 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1)
       self.conv256_1 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
    #    self.conv256_2 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1)
    #    self.conv256_3 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1)
       self.conv512_1 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1)
    #    self.conv512_2 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1)
    #    self.conv512_3 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1)
    #    self.conv512_4 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1)
    #    self.conv512_5 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1)
    #    self.conv512_6 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1)

       self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
       self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
       self.fc1 = nn.Linear(512, 4096)
       self.fc2 = nn.Linear(4096,1000)
       self.fc3 = nn.Linear(1000, num_classes)

    def forward(self,x):
        x = f.relu(self.conv64_1(x))
        x = self.pool(x)
        x = f.relu(self.conv128_1(x))
        x = self.pool(x)
        x = f.relu(self.conv256_1(x))
        x = self.pool(x)
        x = f.relu(self.conv512_1(x))
        x = self.pool(x)

        x = self.global_avg_pool(x)
        
        x = x.view(x.size(0), -1)
        x = f.relu(self.fc1(x))
        x = f.relu(self.fc2(x))
        x = f.softmax(self.fc3(x), dim=1)

        return x

In [9]:
#Initialize network
model = VGG16C().to(device)
#loss function network
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

In [ ]:
# Train and Network
losses = []
for epoch in range(num_epoch):
    for batch_idx,(data,targets) in enumerate(train_data_loader):
        data = data.to(device=device)
        targets = targets.to(device=device)

        #forward
        score = model(data)
        loss = criterion(score,targets)

        #backward
        optimizer.zero_grad()
        loss.backward()

        #gradient descent or adam step
        optimizer.step()

    losses.append(loss.item())
    #mean_loss = sum(losses) / len(losses)
    #print(f'epoch: {epoch+1}/{num_epoch } loss =  {mean_loss:.5f}')
    print('Epoch [{}/{}], loss:{:4f}'.format(epoch+1,num_epoch,loss.item()))

plt.plot(losses,label="train")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [10]:
def check_accuracy(loader, model):

  num_correct = 0
  num_samples = 0
  model.eval() # -> Evaluate for model


  with torch.no_grad():
    for x, y in loader:
      #when train on GPU
      x=x.type(torch.cuda.FloatTensor)
      y=y.type(torch.cuda.FloatTensor)

      #CPU
      #x = x.to(device=device)
      #y = y.to(device=device)


      scores = model(x) # CNN
      _, predictions = scores.max(1)
      num_correct += (predictions==y).sum()
      num_samples += predictions.size(0)

  print(f'got {num_correct} / {num_samples}   Accuracy {(float(num_correct)/float(num_samples)*100)}')

  model.train()

In [ ]:
check_accuracy(train_data_loader, model)
check_accuracy(test_data_loader, model)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import numpy as np


y_pred = []
y_true = []

# iterate over test data
for inputs, labels in train_data_loader:
        #when train on GPU
        inputs=inputs.type(torch.cuda.FloatTensor)
        labels=labels.type(torch.cuda.FloatTensor)


        output = model(inputs) # Feed Network
        output = (torch.max(torch.exp(output), 1)[1]).data.cpu().numpy()
        #output = output.reshape(output.shape[0], -1)
        y_pred.extend(output) # Save Prediction

        labels = labels.data.cpu().numpy()
        y_true.extend(labels) # Save Truth

# constant for classes
classes = ('glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor')

# Build confusion matrix
cf_matrix = confusion_matrix(y_true, y_pred)
df_cm = pd.DataFrame(cf_matrix/np.sum(cf_matrix) *10, index = [i for i in classes],
                     columns = [i for i in classes])
plt.figure(figsize = (12,7))
sn.heatmap(df_cm, annot=True)
plt.savefig('/ConfMatrix/confusion.png')